In [ ]:
# install a pip package in the current jupyter kernel
import sys
!{sys.executable} -m pip install arm-pyart matplotlib
!{sys.executable} -m pip install cartopy boto3
!{sys.executable} -m pip install botocore
#!{sys.executable} -m pip install botocore==1.27.60
#!{sys.executable} -m pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached botocore-1.27.59-py3-none-any.whl (9.1 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.13
    Uninstalling botocore-1.29.13:
      Successfully uninstalled botocore-1.29.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
boto3 1.26.13 requires botocore<1.30.0,>=1.29.13, but you have botocore 1.27.59 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached botocore-1.29.13-py3-none-any.whl (9.9 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.27.59
    Uninstalling botocore-1.27.59:
      Successfully uninst

In [ ]:

import pyart as pa

import pandas as pd

import boto3
import botocore
from botocore import UNSIGNED
from botocore.client import Config

import cartopy.crs as ccrs
import matplotlib.pyplot as plt

debug = True

stations = ["KABR", "KIND", "KJKL", "KDHL"]
months = ["11", "12", "01", "02", "03"]
days = ["01", "02", "03", "04", "05", "06", "07", "08", "09"]
for day in range(10,32):
  days.append(day)

if debug:
  # less data to test
  stations = ["KIND"]
  months = ["11"]
  days = ["01"]

bucket_links = []

s3 = boto3.resource('s3', config = Config(signature_version = botocore.UNSIGNED, user_agent_extra = 'Resource'))
bucket = s3.Bucket('noaa-nexrad-level2')
for station in stations:
  for month in months:
    for day in days:
      for year in ['2019', '2020']:
        for obj in bucket.objects.filter(Prefix = f'{year}/{month}/{day}/{station}/{station}{year}'):
          if not ("MDM" in obj.key):
            bucket_links.append(f's3://noaa-nexrad-level2/{obj.key}')

KABR = []
KIND = []
KJKL = []
KDHL = []
aws_nexrad_level2_file = "s3://noaa-nexrad-level2/2022/03/22/KHGX/KHGX20220322_120125_V06"
radar2 = pa.io.read_nexrad_archive(aws_nexrad_level2_file)

for link in bucket_links:
  print(link)
  radar = pyart.io.read_nexrad_archive(link)
  if "KABR" in link:
    KABR.append(radar)
  elif "KIND" in link:
    KIND.append(radar)
  elif "KJKL" in link:
    KJKL.append(radar)
  else:
    KDHL.append(radar)
  if debug: 
    break

print(KABR)
print(KIND)
print(KJKL)
print(KDHL)


PermissionError: ignored

In [ ]:
import pyart

if pyart.__version__ == "1.7.0":
    if hasattr(pyart.graph, 'RadarMapDisplay'):
        print("Ready to go")
    else:
        print ("ISSUES\n\nMissing\Broken Basemap\n")
        pyart._debug_info()
else:
    print ("ISSUES\n\nWrong Py-ART version\n")
    pyart._debug_info()

<>:7: DeprecationWarning: invalid escape sequence \B
<>:7: DeprecationWarning: invalid escape sequence \B
<>:7: DeprecationWarning: invalid escape sequence \B
<ipython-input-19-d06136d405d6>:7: DeprecationWarning: invalid escape sequence \B
  print ("ISSUES\n\nMissing\Broken Basemap\n")


ISSUES

Wrong Py-ART version

Py-ART version: 1.14.1

---- Dependencies ----
Python version: 3.7.15
NumPy version: 1.21.6
SciPy version: 1.7.3
matplotlib version: 3.2.2
netCDF4 version: 1.6.1

---- Optional dependencies ----
CyLP: MISSING
PyGLPK version: MISSING
CVXOPT version: 1.3.0
Cartopy version: 0.19.0.post1
pytest version: 3.6.4


In [ ]:
# EXAMPLE CLASS BLOCK


nr_file = "s3://noaa-nexrad-level2/2019/11/01/KABR/KABR20191101_111308_V06"

import pyart

radar = pyart.io.read_nexrad_archive(nr_file)

list(radar.fields)




In [ ]:
# Iterate through allscreening folder
# https://www.geeksforgeeks.org/how-to-iterate-over-files-in-directory-using-python/

# import required module
import os
# assign directory
directory = 'allscreenings'
 
# iterate over files in
# that directory
for filename in os.listdir(directory):



    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)

